# Examen Parcial TNUI 2016  

Cal entregar aquest mateix notebook "PrimerParcial2016_NomCognom.ipynb" amb els exercicis completats

### Càrrega de dades

No cal realitzar cap modificació en la càrrega de dades. Ja es redueix el dataset per poder treballar-hi còmodament.

In [108]:
# lectura de dades
import pandas as pd
import numpy as np

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

data = pd.merge(pd.merge(ratings, users), movies)

In [109]:
# seleccionem un subconjunt de les dades per treballar més ràpid
np.random.seed(42)
movielens = data.ix[np.random.choice(data.index, size=10000, replace=False)]
print 'Usuaris:', movielens.user_id.nunique()
print 'Films:',movielens.movie_id.nunique()
# seleccionem les files dels usuaris que tenen al menys 2 ratings

user_ids_larger_1 = pd.value_counts(movielens.user_id, sort=False) > 1
movielens = movielens[user_ids_larger_1[movielens.user_id].values]
print movielens.shape

Usuaris: 3699
Films: 2318
(8462, 10)


In [110]:
# generem subconjunts de training i test
import numpy as np

def assign_to_set(df):
    np.random.seed(42)
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

movielens['for_testing'] = False
grouped = movielens.groupby('user_id', group_keys=False).apply(assign_to_set)
movielens_train = movielens[grouped.for_testing == False]
movielens_test = movielens[grouped.for_testing == True]
print movielens_train.shape
print movielens_test.shape
print movielens_train.tail()

(5831, 11)
(2631, 11)
        user_id  movie_id  rating  timestamp gender  age  occupation    zip  \
708108     2899      3268       1  971918661      M   35          14  94133   
399844      429      3160       5  976255718      M   18           0  54901   
309922     1962      3051       1  974686857      F   18           1  60914   
905326     1941      2057       4  974854645      M   35          17  94550   
230000     5026        39       3  962578775      M   25          17  92584   

                                    title                genres for_testing  
708108  Stop! Or My Mom Will Shoot (1992)         Action|Comedy       False  
399844                    Magnolia (1999)                 Drama       False  
309922           Anywhere But Here (1999)                 Drama       False  
905326     Incredible Journey, The (1963)  Adventure|Children's       False  
230000                    Clueless (1995)        Comedy|Romance       False  


In [111]:
# definim una funció per avaluar el resultat de la recomanació.
def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate):
    ids_to_estimate = zip(movielens_test['user_id'], movielens_test['movie_id'])
    estimated = np.array([estimate(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

## Exemple 

Avaluació d'un a política de recomanació d'un ítem a un usuari basada en donar aleatòriament un a puntuació entre 1 i 5

In [112]:
def rec1(user_id, item_id,train=movielens_train):
    return np.random.choice(range(1,6))

print 'Error: %f' % evaluate(rec1)

Error: 1.883635


## Exercici 1

Quin és el codi postal (columna ``zip``) dels usuaris que té la puntuació mitja més baixa al conjunt de training?

In [113]:
# Escriu la solució aquí.
lowest_mean = (100.0, None)
ratings = pd.DataFrame(movielens_train[['zip','rating']])
for code in ratings['zip']: 
    low = ratings[ratings['zip'] == code]['rating'].mean()
    if low < lowest_mean[0]:
        lowest_mean = (low, code)
print lowest_mean

(1.0, '78734')


## Exercici 2

Avalua la política de recomanació basada en assignar a un ítem la mitja de les recomanacions dels altres usuaris, independentment de la seva semblança. Si no podem fer cap recomanació, retorna 3. Quin és el valor RMSE? Quin és el rating estimat de l'usuari 5988 per l'ítem 11? 

In [114]:
# Escriu la solució aquí.
def rec2(user_id, item_id,train=movielens_train):
    ratings = pd.DataFrame(train[train['movie_id'] == item_id][['user_id','rating']])
    
    for i in ratings['user_id']:
        if i == user_id:
            print "ELIMINATING MYSELF"
            ratings = ratings.drop(user_id)
            
    if len(ratings) == 0:
        return 3
    
    return ratings['rating'].mean()

print 'Error: %f' % evaluate(rec2)
print 'Estimació per a l\'usuari %d sobre l\'ítem %d : %f' % (5988,11,rec2(5988,11))

Error: 1.124096
Estimació per a l'usuari 5988 sobre l'ítem 11 : 3.000000


## Exercici 3

Modifica el recomanador anterior de manera que només faci servir usuaris del mateix rang d'edats que el de l'usuari per recomanar-li una pel·lícula. Si no podem fer cap recomanació, retorna 3. Quin és el valor RMSE? Quin rating estimat de l'usuari 5988 per l'ítem 11? 

In [115]:
# Escriu la solució aquí.
def rec3(user_id, item_id,train=movielens_train):
    ratings = pd.DataFrame(train[train['movie_id'] == item_id][['user_id','rating','age']])
    age_range = train[train['user_id'] == user_id]['age']
    age_range.index = range(len(age_range))
    
    try:
        ratings = ratings[ratings['age'] == age_range[0]]
    except:
        return 3 #No users within the same age_range left
    
    for i in ratings['user_id']:
        if i == user_id:
            print "ELIMINATING MYSELF"
            ratings = ratings.drop(user_id)

    if len(ratings) == 0:
        return 3
 
    return ratings['rating'].mean()

print 'Error: %f' % evaluate(rec3)
print 'Estimació per a l\'usuari %d sobre l\'ítem %d : %f' % (5988,11,rec3(5988,11))

Error: 1.224257
Estimació per a l'usuari 5988 sobre l'ítem 11 : 3.000000


## Exercici 4

Calcula el coeficient de correlació de Pearson entre l'usuari promig dels usuaris de 25 anys i l'usuari promig dels usuaris de 50 en el conjunt d'aprenentatge, considerant els films pels quals tenim puntuació de les dues franges d'edat.

In [118]:
# Escriu la solució aquí.
from scipy.stats import pearsonr

table_25 = movielens_train[movielens_train['age'] == 25][['movie_id','rating']]
table_50 = movielens_train[movielens_train['age'] == 50][['movie_id','rating']]
table = pd.merge(table_25,table_50,on='movie_id')

rate_x = [0] * len(table) #Saving mean movie ratings from age_range '25' here
rate_y = [0] * len(table) #Saving mean movie ratings from age_range '50' here
for i, m in zip(range(len(table)), table['movie_id']):
    rate_x[i] = table[table['movie_id'] == m]['rating_x'].mean()
    rate_y[i] = table[table['movie_id'] == m]['rating_y'].mean()

# Càlcul de la correlació de Pearson
p_coeff = pearsonr(rate_x, rate_y)
print "Pearson's coeff: [", p_coeff[0], "]", p_coeff 

Pearson's coeff: [ 0.205994888988 ] (0.20599488898790472, 1.5244977815387076e-08)
